# How does Julia's performance compare to C and Python?

Let's look at and benchmark the sum function:

$$\mathrm{sum}(x) = \sum_{i=1}^n x_i$$

In [ ]:
x = rand(10^7);

In [ ]:
sum(x)

In [ ]:
d = Dict() # to store the measurement results

# Python

In [ ]:
using BenchmarkTools
using PyCall

## numpy

In [ ]:
np = pyimport("numpy")

In [ ]:
numpy_sum = np.sum

In [ ]:
b = @benchmark $numpy_sum($x)

In [ ]:
d["Python (numpy)"] = minimum(b.times) / 1e6

## hand-written

In [ ]:
py"""
def mysum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

In [ ]:
mysum_py = py"mysum"

In [ ]:
# call a low-level PyCall function to get a Python list, because
# by default PyCall will convert to a NumPy array instead (we benchmark NumPy below):
xpy_list = PyCall.array2py(x);

In [ ]:
b = @benchmark $mysum_py($xpy_list)

In [ ]:
d["Python (hand-written)"] = minimum(b.times) / 1e6

## built-in

In [ ]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

In [ ]:
b = @benchmark $pysum($xpy_list)

In [ ]:
d["Python (built-in)"] = minimum(b.times) / 1e6

# C

## hand-written

In [ ]:
c_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
""";

In [ ]:
# compile to a shared library by piping C_code to gcc:
# (only works if you have gcc installed)
const Clib = tempname()
using Libdl

In [ ]:
open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, c_code)
end

In [ ]:
c_sum(X::Array{Float64}) = @ccall Clib.c_sum(length(X)::Csize_t, X::Ptr{Float64})::Float64

In [ ]:
c_sum(x) ≈ sum(x)

In [ ]:
b = @benchmark c_sum($x)

In [ ]:
d["C"] = minimum(b.times) / 1e6

## hand-written (with `-fast-math`)

In [ ]:
const Clib_fastmath = tempname()   # make a temporary file

# The same as above but with a -ffast-math flag added
open(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o $(Clib_fastmath * "." * Libdl.dlext) -`, "w") do f
    print(f, c_code) 
end

# define a Julia function that calls the C function:
c_sum_fastmath(X::Array{Float64}) = @ccall Clib_fastmath.c_sum(length(X)::Csize_t, X::Ptr{Float64})::Float64

In [ ]:
b = @benchmark c_sum_fastmath($x)

In [ ]:
d["C (fastmath)"] = minimum(b.times) / 1e6

# Julia

## built-in

In [ ]:
b = @benchmark sum($x)

In [ ]:
d["Julia (built-in)"] = minimum(b.times) / 1e6

In [ ]:
@benchmark reduce($+, $x)

## built-in (with `Vector{Any}`)

In [ ]:
x_any = Vector{Any}(x)
b = @benchmark sum($x_any)

In [ ]:
d["Julia (built-in, Any)"] = minimum(b.times) / 1e6

## hand-written

In [ ]:
function mysum(A)
    s = zero(eltype(A)) # the correct type of zero for A
    for a in A
        s += a
    end
    return s
end

In [ ]:
b = @benchmark mysum($x)

In [ ]:
d["Julia (hand-written)"] = minimum(b.times) / 1e6

## hand-written (with `@simd`)

In [ ]:
function mysum_simd(A)
    s = zero(eltype(A)) # the correct type of zero for A
    @simd for a in A
        s += a
    end
    return s
end

In [ ]:
b = @benchmark mysum_simd($x)

In [ ]:
d["Julia (hand-written, simd)"] = minimum(b.times) / 1e6

## Summary

In [ ]:
for (key, value) in sort(collect(d), by=x->x[2])
    println(rpad(key, 30, "."), lpad(round(value, digits=2), 10, "."))
end

And of course, our hand-written Julia implementation is type-generic!

# Supplement: What about other functions?

### Log

In [ ]:
@which log(1.2)

In [ ]:
using BenchmarkTools

# uses the system C library
clog(x) = ccall(:log, Float64, (Float64,), x)
# uses LLVM's log
llvmlog(x) =  ccall(Symbol("llvm.log.f64"), llvmcall, Float64, (Float64,), x)

@btime log($(Ref(1.2))[])    
@btime clog($(Ref(1.2))[])    
@btime llvmlog($(Ref(1.2))[]);

On my Windows 10 machine:

```
  6.199 ns (0 allocations: 0 bytes)
  9.408 ns (0 allocations: 0 bytes)
  8.599 ns (0 allocations: 0 bytes)
```

### Exp

In [ ]:
@which exp(1.2)

In [ ]:
using BenchmarkTools

# uses the system C library
cexp(x) = ccall(:exp, Float64, (Float64,), x)
# uses LLVM's
llvmexp(x) =  ccall(Symbol("llvm.exp.f64"), llvmcall, Float64, (Float64,), x)

@btime exp($(Ref(1.2))[])    
@btime cexp($(Ref(1.2))[])    
@btime llvmexp($(Ref(1.2))[]);

On my Windows 10 machine:

```
  7.899 ns (0 allocations: 0 bytes)
  22.870 ns (0 allocations: 0 bytes)
  22.868 ns (0 allocations: 0 bytes)
```

### Matrix multiplication

In [ ]:
N = 10
C = zeros(N,N);
A = rand(N,N);
B = rand(N,N);

In [ ]:
using LinearAlgebra

mul!(C, A, B); # "built-in", calls underlying BLAS/LAPACK

In [ ]:
C ≈ A * B

In [ ]:
using BenchmarkTools

function mul_naive!(𝐂, 𝐀, 𝐁)
   @inbounds for m ∈ axes(𝐀,1), n ∈ axes(𝐁,2)
       𝐂mn = zero(eltype(𝐂))
       for k ∈ axes(𝐀,2)
           𝐂mn += 𝐀[m,k] * 𝐁[k,n]
       end
       𝐂[m,n] = 𝐂mn
   end
end

In [ ]:
mul_naive!(C,A,B)
C ≈ A * B

[LoopVectorization.jl](https://github.com/chriselrod/LoopVectorization.jl)

In [ ]:
using LoopVectorization

function mul_avx!(𝐂, 𝐀, 𝐁)
   @avx for m ∈ axes(𝐀,1), n ∈ axes(𝐁,2)
       𝐂mn = zero(eltype(𝐂))
       for k ∈ axes(𝐀,2)
           𝐂mn += 𝐀[m,k] * 𝐁[k,n]
       end
       𝐂[m,n] = 𝐂mn
   end
end

In [ ]:
mul_avx!(C,A,B)
C ≈ A * B

In [ ]:
c_code = """
#include <stddef.h>
#include <math.h>

void gemm(double* restrict C, double* restrict A, double* restrict B, long M, long K, long N){
  for (long i = 0; i < M*N; i++){
    C[i] = 0.0;
  }
  for (long n = 0; n < N; n++){
    for (long k = 0; k < K; k++){
      for (long m = 0; m < M; m++){
	C[m + n*M] += A[m + k*M] * B[k + n*K];
      }
    }
  }
  return;
}
""";

In [ ]:
# compile to a shared library by piping C_code to gcc:
# (only works if you have gcc installed)
const Clib_gemm = tempname()
using Libdl

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib_gemm * "." * Libdl.dlext) -`, "w") do f
    print(f, c_code)
end

c_gemm(C::Array{Float64},A::Array{Float64},B::Array{Float64}) = @ccall Clib_gemm.gemm(C::Ptr{Float64}, A::Ptr{Float64}, B::Ptr{Float64}, size(A,1)::Clong, size(A,2)::Clong, size(B,2)::Clong)::Cvoid

In [ ]:
c_gemm(C, A, B)
C ≈ A*B

In [ ]:
@btime mul!($C, $A, $B);
@btime mul_naive!($C,$A,$B);
@btime mul_avx!($C,$A,$B);
@btime c_gemm($C2, $A, $B)

**Note for larger `N`:** BLAS is multithreaded for larger `N`. In this case our `mul_avx!` is slower than `mul!`.

In [ ]:
N = 100
C = zeros(N,N);
A = rand(N,N);
B = rand(N,N);

In [ ]:
@btime mul_avx!($C, $A, $B);
@btime mul!($C, $A, $B);

However, [Octavian.jl](https://github.com/JuliaLinearAlgebra/Octavian.jl) is a (experimental) package that adds multithreading on top of LoopVectorization.jl. Again, we can blast OpenBLAS out of the water with pure Julia code. (Restart the kernel before executing the following.)

In [ ]:
Threads.nthreads()

In [ ]:
using Pkg
pkg"activate --temp"
pkg"add Octavian"
pkg"add BenchmarkTools"

In [ ]:
using Octavian, LinearAlgebra, BenchmarkTools

In [ ]:
N = 100; C = zeros(N,N); A = rand(N,N); B = rand(N,N);

In [ ]:
@btime Octavian.matmul!($C, $A, $B);

In [ ]:
@btime mul!($C, $A, $B);

**Resources/more info:**

https://github.com/mitmath/18S096/blob/master/lectures/lecture1/Performance-variation.ipynb

https://github.com/mitmath/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb


**More comprehensive benchmarks:** https://chriselrod.github.io/LoopVectorization.jl/stable/examples/matrix_multiplication/#Matrix-Multiplication